<a href="https://colab.research.google.com/github/eeroolli/SalsaAnnotation/blob/development/data/process/video_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
import pandas as pd
import numpy as np
import cv2, os
import csv

# libraries needed from outside
import os
from pathlib import Path
from os import mkdir
from os.path import exists, join, basename, splitext
import sys
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
#import shutil                     # used for copying files in the os.
import pandas as pd
!pip install validators
import validators                 #for URL validation
#from validators import ValidationFailure


from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [227]:
root_path = "/content/gdrive/MyDrive/DSR/DanceApp/"

#root_path = "~/Documents/SalsaAnnotation/"
input_csv = os.path.join(root_path, "ml/models/Data_train_validate/Data_train_norm.csv")
data_output_path = os.path.join(root_path,"Model-Test", "Data_train_validate")
script_path = "/content/gdrive/MyDrive/DSR/2021-DSR-Porfolio/python_scripts"
# one cannot add items to sys.path that have a space in them and script_path must be
# the same for Ana and Eero.  Eero has now added a link from his google DSR to Ana's google 2021-DSR-Porfolio/

print("Using ", root_path, " as root_path")
print("Using ", analysis_path, " as analysis_path")
print("Using ", input_data, " as input_data")

def check_path(x):
  if os.path.isdir(x):
    print("Using", x )
  else:
    print("Problem: There was no ", x)  # No error keeps running

check_path(root_path)

def check_url(url_string: str):
    if validators.url(url_string):
      print("Using: ", url_string)
    else:
      print("Problem: The url is not valid ", url_string) 
      return False

# to select clip_names
video_size = "_h920"  # use format _h920


Using  /content/gdrive/MyDrive/DSR/DanceApp/  as root_path
Using  ~/Documents/SalsaAnnotation/exploratory  as analysis_path
Using  https://github.com/eeroolli/SalsaAnnotation/ml/models/Data_train_validate/Data_train_norm.csv  as input_data
Using /content/gdrive/MyDrive/DSR/DanceApp/


In [211]:
# get pose data - data is generated by open pose video
check_url(input_csv)

df = pd.read_csv("https://raw.githubusercontent.com/eeroolli/SalsaAnnotation/development/ml/models/Data_train_validate/Data_train_norm.csv")
df_original = df.copy()
df


Problem: The url is not valid  /content/gdrive/MyDrive/DSR/DanceApp/ml/models/Data_train_validate/Data_train_norm.csv


,Unnamed: 0,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,0,1P-Person1-whole,1088,0.0,0.681696,0.853538,0.737357,0.207688,0.833340,0.141048,0.659949,0.285909,0.430329,0.282942,0.725462,0.268262,0.650518,0.370922,0.497119,0.430552,0.738171,0.126585,0.799872,0.148094,0.744685,0.201244,0.749583,0.450522,0.723299,0.153962,0.615481,0.167664,0.626775,0.423745,0.854251,0.925021,0.719415,0.881862,0.789561,0.240254,...,0.514523,0.370306,0.512039,0.437526,0.645643,0.459093,0.682073,0.404360,0.694716,0.407190,0.749492,0.509332,0.940791,0.825485,0.756956,0.220520,0.428242,0.828087,0.879523,0.839556,0.749144,0.723278,0.869167,0.832241,0.755291,0.820028,0.869713,0.0,0.883145,0.0,0.910062,0.722660,0.741301,0.726582,0.357497,0.268698,0.845678,basic,S,331
1,1,1P-Person1-whole,1089,0.0,0.675229,0.853538,0.730654,0.212059,0.822882,0.133202,0.649326,0.292344,0.421296,0.292281,0.716025,0.275384,0.638838,0.401118,0.484275,0.375760,0.726028,0.124798,0.792038,0.148320,0.735739,0.202195,0.748170,0.451996,0.710187,0.154352,0.617658,0.178516,0.619056,0.427081,0.854417,0.924955,0.707452,0.882272,0.790770,0.240263,...,0.501582,0.368315,0.499469,0.440602,0.637301,0.467313,0.689297,0.407921,0.714213,0.408095,0.748797,0.516031,0.961686,0.819557,0.746936,0.061383,0.114665,0.843735,0.840574,0.853456,0.740071,0.718373,0.860143,0.846571,0.743226,0.818874,0.867462,0.0,0.875510,0.0,0.897884,0.695361,0.729914,0.749858,0.330250,0.268070,0.859535,basic,NaN,331
2,2,1P-Person1-whole,1090,0.0,0.671985,0.853556,0.728756,0.227185,0.810413,0.134718,0.660899,0.315080,0.425772,0.326848,0.711026,0.287274,0.627417,0.432800,0.469012,0.337169,0.713574,0.127847,0.782186,0.151402,0.722631,0.205664,0.744324,0.454308,0.694580,0.153523,0.622360,0.196573,0.617845,0.431830,0.854583,0.924889,0.693877,0.884043,0.791978,0.240271,...,0.494878,0.370560,0.490243,0.446010,0.635306,0.475823,0.689124,0.413454,0.721847,0.412606,0.744742,0.517501,0.964532,0.821165,0.733104,0.000000,0.000000,0.863146,0.840684,0.845682,0.745623,0.730360,0.835461,0.848445,0.740062,0.859034,0.885281,0.0,0.882117,0.0,0.908143,0.742254,0.773109,0.752221,0.355033,0.280502,0.854643,basic,NaN,331
3,3,1P-Person1-whole,1091,0.0,0.666657,0.853571,0.725627,0.248969,0.794781,0.145861,0.685055,0.348644,0.487102,0.452482,0.706756,0.301061,0.615728,0.465800,0.456172,0.312228,0.704522,0.128129,0.770890,0.149685,0.704558,0.206496,0.736037,0.452645,0.682166,0.150387,0.629394,0.211126,0.620391,0.435076,0.854748,0.924824,0.682021,0.885384,0.793186,0.240280,...,0.493611,0.373032,0.486122,0.453750,0.634166,0.481717,0.688210,0.415755,0.717877,0.419330,0.735906,0.507632,0.972222,0.816849,0.731502,0.112481,0.252638,0.872159,0.901609,0.862774,0.777462,0.748957,0.852879,0.859418,0.773417,0.892421,0.892906,0.0,0.881318,0.0,0.914858,0.722774,0.747182,0.724742,0.387913,0.309943,0.841137,basic,NaN,331
4,4,1P-Person1-whole,1092,0.0,0.662175,0.853559,0.721041,0.272874,0.780143,0.160763,0.692836,0.390540,0.513351,0.526399,0.706018,0.312849,0.600512,0.493168,0.438967,0.293378,0.693668,0.130602,0.755526,0.148066,0.680629,0.206570,0.728227,0.449337,0.673086,0.149922,0.635515,0.221716,0.621151,0.440013,0.854914,0.924758,0.671675,0.886570,0.794395,0.240289,...,0.494506,0.376412,0.487880,0.459346,0.633823,0.487772,0.682594,0.417223,0.704779,0.420062,0.724590,0.493466,0.970407,0.815011,0.711973,0.000000,0.000000,

In [212]:
df.columns

Index(['Unnamed: 0', 'clip_name', 'frame_nr', 'person_id', 'nose_x', 'nose_y',
       'neck_x', 'neck_y', 'rshoulder_x', 'rshoulder_y', 'relbow_x',
       'relbow_y', 'rwrist_x', 'rwrist_y', 'lshoulder_x', 'lshoulder_y',
       'lelbow_x', 'lelbow_y', 'lwrist_x', 'lwrist_y', 'midhip_x', 'midhip_y',
       'rhip_x', 'rhip_y', 'rknee_x', 'rknee_y', 'rankle_x', 'rankle_y',
       'lhip_x', 'lhip_y', 'lknee_x', 'lknee_y', 'lankle_x', 'lankle_y',
       'reye_x', 'reye_y', 'leye_x', 'leye_y', 'rear_x', 'rear_y', 'lear_x',
       'lear_y', 'lbigtoe_x', 'lbigtoe_y', 'lsmalltoe_x', 'lsmalltoe_y',
       'lheal_x', 'lheal_y', 'rbigtoe_x', 'rbigtoe_y', 'rsmalltoe_x',
       'rsmalltoe_y', 'rheal_x', 'rheal_y', 'nose_v', 'neck_v', 'rshoulder_v',
       'relbow_v', 'rwrist_v', 'lshoulder_v', 'lelbow_v', 'lwrist_v',
       'midhip_v', 'rhip_v', 'rknee_v', 'rankle_v', 'lhip_v', 'lknee_v',
       'lankle_v', 'reye_v', 'leye_v', 'rear_v', 'lear_v', 'lbigtoe_v',
       'lsmalltoe_v', 'lheal_v', 'rbigto

## Flip on X axis

Flip json. Use normalized data x = 1 - x



In [213]:
#df.columns.str.endswith('_x')

In [214]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#          Data Augmentation -- Lists of column names
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++

pose_x_list = ['nose_x', 'neck_x', 'rshoulder_x', 'relbow_x', 'rwrist_x', 'lshoulder_x', 'lelbow_x', 'lwrist_x', 
              'midhip_x', 'rhip_x', 'rknee_x', 'rankle_x', 'lhip_x', 'lknee_x', 'lankle_x', 
              'reye_x', 'leye_x',  'rear_x', 'lear_x',
              'lbigtoe_x', 'lsmalltoe_x', 'lheal_x', 'rbigtoe_x', 'rsmalltoe_x', 'rheal_x' ]

pose_v_list = []
pose_y_list = []
for x_var in pose_x_list:
   pose_y_list.append(x_var.replace('_x', '_y')) 
   pose_v_list.append(x_var.replace('_x', '_v')) 

pose_xy_list = pose_x_list + pose_y_list
print(pose_xy_list)

['nose_x', 'neck_x', 'rshoulder_x', 'relbow_x', 'rwrist_x', 'lshoulder_x', 'lelbow_x', 'lwrist_x', 'midhip_x', 'rhip_x', 'rknee_x', 'rankle_x', 'lhip_x', 'lknee_x', 'lankle_x', 'reye_x', 'leye_x', 'rear_x', 'lear_x', 'lbigtoe_x', 'lsmalltoe_x', 'lheal_x', 'rbigtoe_x', 'rsmalltoe_x', 'rheal_x', 'nose_y', 'neck_y', 'rshoulder_y', 'relbow_y', 'rwrist_y', 'lshoulder_y', 'lelbow_y', 'lwrist_y', 'midhip_y', 'rhip_y', 'rknee_y', 'rankle_y', 'lhip_y', 'lknee_y', 'lankle_y', 'reye_y', 'leye_y', 'rear_y', 'lear_y', 'lbigtoe_y', 'lsmalltoe_y', 'lheal_y', 'rbigtoe_y', 'rsmalltoe_y', 'rheal_y']


In [215]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#          Data Augmentation -- Horisontal Flip
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++

import numpy as np
np.set_printoptions(precision=2)  # For compact display.
df_flipped = df_original.copy()
             

for var in pose_x_list:
  df_flipped[var] = 1 - df_original[var]


In [216]:
print(df_flipped.nose_x.describe())
print(df_original.nose_x.describe())

count    35922.000000
mean         0.466235
std          0.199172
min          0.000000
25%          0.321266
50%          0.469943
75%          0.600338
max          1.000000
Name: nose_x, dtype: float64
count    35922.000000
mean         0.533765
std          0.199172
min          0.000000
25%          0.399662
50%          0.530057
75%          0.678734
max          1.000000
Name: nose_x, dtype: float64


In [230]:
## Saving the results

data_output_csv = os.path.join(data_output_path, "Data_train_flipped_x.csv")
df_perspective1_norm.to_csv(data_output_csv)




## perspective change. 

Use normalized data:
Y stays the same.
X gets smaller in the top of the frame. 

Top of the frame Y = 0 
Bottom = 1. If x is reduced by 50% on top of the frame.  Bottom is multiplied with 1-Y. Top is multiplied with 0.5  multiplicator goes from [0, 0.5] when y goes from [1,0]


In [217]:
#+++++++++++++++++++++++++++++++++++++++++++++
#   Data Augmentation - Perspective Change 1
#   Reduce the width on top 
# ++++++++++++++++++++++++++++++++++++++++++++

reduction_factor = 0.4
df_perspective1 = df_original.copy()

for x_var in pose_x_list:
  y_var = x_var.replace('_x', '_y')
  # print(x_var, y_var)
  df_perspective1[x_var] = (1- (reduction_factor * (1- df_original[y_var]))) * df_original[var]



In [218]:
print(df_perspective1.nose_x.describe())
print(df_original.nose_x.describe())

count    35922.000000
mean         0.389733
std          0.142782
min          0.000000
25%          0.295163
50%          0.384943
75%          0.473111
max          0.908174
Name: nose_x, dtype: float64
count    35922.000000
mean         0.533765
std          0.199172
min          0.000000
25%          0.399662
50%          0.530057
75%          0.678734
max          1.000000
Name: nose_x, dtype: float64


In [219]:
df_original.head(-10)

,Unnamed: 0,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,0,1P-Person1-whole,1088,0.0,0.681696,0.853538,0.737357,0.207688,0.833340,0.141048,0.659949,0.285909,0.430329,0.282942,0.725462,0.268262,0.650518,0.370922,0.497119,0.430552,0.738171,0.126585,0.799872,0.148094,0.744685,0.201244,0.749583,0.450522,0.723299,0.153962,0.615481,0.167664,0.626775,0.423745,0.854251,0.925021,0.719415,0.881862,0.789561,0.240254,...,0.514523,0.370306,0.512039,0.437526,0.645643,0.459093,0.682073,0.404360,0.694716,0.407190,0.749492,0.509332,0.940791,0.825485,0.756956,0.220520,0.428242,0.828087,0.879523,0.839556,0.749144,0.723278,0.869167,0.832241,0.755291,0.820028,0.869713,0.0,0.883145,0.0,0.910062,0.722660,0.741301,0.726582,0.357497,0.268698,0.845678,basic,S,331
1,1,1P-Person1-whole,1089,0.0,0.675229,0.853538,0.730654,0.212059,0.822882,0.133202,0.649326,0.292344,0.421296,0.292281,0.716025,0.275384,0.638838,0.401118,0.484275,0.375760,0.726028,0.124798,0.792038,0.148320,0.735739,0.202195,0.748170,0.451996,0.710187,0.154352,0.617658,0.178516,0.619056,0.427081,0.854417,0.924955,0.707452,0.882272,0.790770,0.240263,...,0.501582,0.368315,0.499469,0.440602,0.637301,0.467313,0.689297,0.407921,0.714213,0.408095,0.748797,0.516031,0.961686,0.819557,0.746936,0.061383,0.114665,0.843735,0.840574,0.853456,0.740071,0.718373,0.860143,0.846571,0.743226,0.818874,0.867462,0.0,0.875510,0.0,0.897884,0.695361,0.729914,0.749858,0.330250,0.268070,0.859535,basic,NaN,331
2,2,1P-Person1-whole,1090,0.0,0.671985,0.853556,0.728756,0.227185,0.810413,0.134718,0.660899,0.315080,0.425772,0.326848,0.711026,0.287274,0.627417,0.432800,0.469012,0.337169,0.713574,0.127847,0.782186,0.151402,0.722631,0.205664,0.744324,0.454308,0.694580,0.153523,0.622360,0.196573,0.617845,0.431830,0.854583,0.924889,0.693877,0.884043,0.791978,0.240271,...,0.494878,0.370560,0.490243,0.446010,0.635306,0.475823,0.689124,0.413454,0.721847,0.412606,0.744742,0.517501,0.964532,0.821165,0.733104,0.000000,0.000000,0.863146,0.840684,0.845682,0.745623,0.730360,0.835461,0.848445,0.740062,0.859034,0.885281,0.0,0.882117,0.0,0.908143,0.742254,0.773109,0.752221,0.355033,0.280502,0.854643,basic,NaN,331
3,3,1P-Person1-whole,1091,0.0,0.666657,0.853571,0.725627,0.248969,0.794781,0.145861,0.685055,0.348644,0.487102,0.452482,0.706756,0.301061,0.615728,0.465800,0.456172,0.312228,0.704522,0.128129,0.770890,0.149685,0.704558,0.206496,0.736037,0.452645,0.682166,0.150387,0.629394,0.211126,0.620391,0.435076,0.854748,0.924824,0.682021,0.885384,0.793186,0.240280,...,0.493611,0.373032,0.486122,0.453750,0.634166,0.481717,0.688210,0.415755,0.717877,0.419330,0.735906,0.507632,0.972222,0.816849,0.731502,0.112481,0.252638,0.872159,0.901609,0.862774,0.777462,0.748957,0.852879,0.859418,0.773417,0.892421,0.892906,0.0,0.881318,0.0,0.914858,0.722774,0.747182,0.724742,0.387913,0.309943,0.841137,basic,NaN,331
4,4,1P-Person1-whole,1092,0.0,0.662175,0.853559,0.721041,0.272874,0.780143,0.160763,0.692836,0.390540,0.513351,0.526399,0.706018,0.312849,0.600512,0.493168,0.438967,0.293378,0.693668,0.130602,0.755526,0.148066,0.680629,0.206570,0.728227,0.449337,0.673086,0.149922,0.635515,0.221716,0.621151,0.440013,0.854914,0.924758,0.671675,0.886570,0.794395,0.240289,...,0.494506,0.376412,0.487880,0.459346,0.633823,0.487772,0.682594,0.417223,0.704779,0.420062,0.724590,0.493466,0.970407,0.815011,0.711973,0.000000,0.000000,

In [220]:
df_perspective1.head(-10)

,Unnamed: 0,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,...,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status,figure_id
0,0,1P-Person1-whole,1088,0.0,0.705583,0.853538,0.511959,0.207688,0.491981,0.141048,0.535410,0.285909,0.534520,0.282942,0.530120,0.268262,0.560897,0.370922,0.578774,0.430552,0.487645,0.126585,0.494093,0.148094,0.510028,0.201244,0.584761,0.450522,0.495853,0.153962,0.499961,0.167664,0.576733,0.423745,0.727014,0.925021,0.714075,0.881862,0.521723,0.240254,...,0.560712,0.370306,0.580864,0.437526,0.587330,0.459093,0.570921,0.404360,0.571770,0.407190,0.602391,0.509332,0.940791,0.825485,0.756956,0.220520,0.428242,0.828087,0.879523,0.839556,0.749144,0.723278,0.869167,0.832241,0.755291,0.820028,0.869713,0.0,0.883145,0.0,0.910062,0.722660,0.741301,0.726582,0.357497,0.268698,0.845678,basic,S,331
1,1,1P-Person1-whole,1089,0.0,0.704929,0.853538,0.512794,0.212059,0.489175,0.133202,0.536841,0.292344,0.536822,0.292281,0.531761,0.275384,0.569421,0.401118,0.561826,0.375760,0.486658,0.124798,0.493703,0.148320,0.509840,0.202195,0.584660,0.451996,0.495510,0.154352,0.502747,0.178516,0.577197,0.427081,0.726320,0.924955,0.713536,0.882272,0.521242,0.240263,...,0.559596,0.368315,0.581247,0.440602,0.589248,0.467313,0.571458,0.407921,0.571511,0.408095,0.603839,0.516031,0.961686,0.819557,0.746936,0.061383,0.114665,0.843735,0.840574,0.853456,0.740071,0.718373,0.860143,0.846571,0.743226,0.818874,0.867462,0.0,0.875510,0.0,0.897884,0.695361,0.729914,0.749858,0.330250,0.268070,0.859535,basic,NaN,331
2,2,1P-Person1-whole,1090,0.0,0.701117,0.853556,0.514523,0.227185,0.486977,0.134718,0.540707,0.315080,0.544212,0.326848,0.532423,0.287274,0.575775,0.432800,0.547287,0.337169,0.484930,0.127847,0.491947,0.151402,0.508112,0.205664,0.582182,0.454308,0.492579,0.153523,0.505404,0.196573,0.575486,0.431830,0.722367,0.924889,0.710199,0.884043,0.518421,0.240271,...,0.557234,0.370560,0.579710,0.446010,0.588591,0.475823,0.570012,0.413454,0.569759,0.412606,0.601007,0.517501,0.964532,0.821165,0.733104,0.000000,0.000000,0.863146,0.840684,0.845682,0.745623,0.730360,0.835461,0.848445,0.740062,0.859034,0.885281,0.0,0.882117,0.0,0.908143,0.742254,0.773109,0.752221,0.355033,0.280502,0.854643,basic,NaN,331
3,3,1P-Person1-whole,1091,0.0,0.692803,0.853571,0.514831,0.248969,0.484479,0.145861,0.544171,0.348644,0.574737,0.452482,0.530164,0.301061,0.578657,0.465800,0.533452,0.312228,0.479260,0.128129,0.485605,0.149685,0.502328,0.206496,0.574785,0.452645,0.485812,0.150387,0.503691,0.211126,0.569614,0.435076,0.713777,0.924824,0.702167,0.885384,0.512273,0.240280,...,0.551350,0.373032,0.575110,0.453750,0.583343,0.481717,0.563926,0.415755,0.564978,0.419330,0.590971,0.507632,0.972222,0.816849,0.731502,0.112481,0.252638,0.872159,0.901609,0.862774,0.777462,0.748957,0.852879,0.859418,0.773417,0.892421,0.892906,0.0,0.881318,0.0,0.914858,0.722774,0.747182,0.724742,0.387913,0.309943,0.841137,basic,NaN,331
4,4,1P-Person1-whole,1092,0.0,0.682147,0.853559,0.513843,0.272874,0.481349,0.160763,0.547947,0.390540,0.587324,0.526399,0.525429,0.312849,0.577692,0.493168,0.519786,0.293378,0.472607,0.130602,0.477669,0.148066,0.494626,0.206570,0.564988,0.449337,0.478207,0.149922,0.499016,0.221716,0.562286,0.440013,0.702783,0.924758,0.691714,0.886570,0.504399,0.240289,...,0.543852,0.376412,0.567889,0.459346,0.576128,0.487772,0.555681,0.417223,0.556503,0.420062,0.577779,0.493466,0.970407,0.815011,0.711973,0.000000,0.000000,

In [221]:
df_perspective1[pose_x_list]

,nose_x,neck_x,rshoulder_x,relbow_x,rwrist_x,lshoulder_x,lelbow_x,lwrist_x,midhip_x,rhip_x,rknee_x,rankle_x,lhip_x,lknee_x,lankle_x,reye_x,leye_x,rear_x,lear_x,lbigtoe_x,lsmalltoe_x,lheal_x,rbigtoe_x,rsmalltoe_x,rheal_x
0,0.705583,0.511959,0.491981,0.535410,0.534520,0.530120,0.560897,0.578774,0.487645,0.494093,0.510028,0.584761,0.495853,0.499961,0.576733,0.727014,0.714075,0.521723,0.473282,0.560712,0.580864,0.587330,0.570921,0.571770,0.602391
1,0.704929,0.512794,0.489175,0.536841,0.536822,0.531761,0.569421,0.561826,0.486658,0.493703,0.509840,0.584660,0.495510,0.502747,0.577197,0.726320,0.713536,0.521242,0.472907,0.559596,0.581247,0.589248,0.571458,0.571511,0.603839
2,0.701117,0.514523,0.486977,0.540707,0.544212,0.532423,0.575775,0.547287,0.484930,0.491947,0.508112,0.582182,0.492579,0.505404,0.575486,0.722367,0.710199,0.518421,0.470507,0.557234,0.579710,0.588591,0.570012,0.569759,0.601007
3,0.692803,0.514831,0.484479,0.544171,0.574737,0.530164,0.578657,0.533452,0.479260,0.485605,0.502328,0.574785,0.485812,0.503691,0.569614,0.713777,0.702167,0.512273,0.465032,0.551350,0.575110,0.583343,0.563926,0.564978,0.590971
4,0.682147,0.513843,0.481349,0.547947,0.587324,0.525429,0.577692,0.519786,0.472607,0.477669,0.494626,0.564988,0.478207,0.499016,0.562286,0.702783,0.691714,0.504399,0.458017,0.543852,0.567889,0.576128,0.555681,0.556503,0.577779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35917,0.508461,0.535551,0.555122,0.609511,0.557427,0.529641,0.570503,0.610923,0.575256,0.556473,0.611569,0.628981,0.593486,0.595352,0.651833,0.497433,0.495482,0.587509,0.588510,0.661575,0.658721,0.677076,0.609472,0.609958,0.657353
35918,0.506609,0.533377,0.549184,0.603528,0.559575,0.528880,0.565418,0.601597,0.572100,0.553829,0.612699,0.625281,0.593580,0.597367,0.661360,0.496096,0.494060,0.584596,0.589117,0.659821,0.664708,0.691309,0.610995,0.611667,0.653306
35919,0.504903,0.533077,0.543164,0.595769,0.566606,0.529768,0.567794,0.591907,0.569506,0.552136,0.612114,0.621691,0.597350,0.599178,0.667844,0.494175,0.492519,0.581006,0.591217,0.653091,0.666914,0.700696,0.607036,0.606976,0.649386
35920,0.502186,0.526878,0.532900,0.569936,0.574898,0.525059,0.573094,0.581933,0.567320,0.548875,0.605607,0.618260,0.596014,0.598477,0.671601,0.491675,0.490256,0.576749,0.589839,0.658456,0.665631,0.704547,0.603097,0.602457,0.644998


In [223]:
# +++++++++++++++++++++++++++++++++++
#    After perspective change we normalize

# this must be run by clip_names
def normalize(array):
  max_value = np.max(array)
  min_value = np.min(array)
  diff = max_value - min_value
  normalized = (array - min_value) / diff
  return normalized

clip_names = df_original['clip_name'].unique()
print(clip_names)

['1P-Person1-whole' '1p_sarah_salsa_basics_choreo1'
 '1p_Eero_4_05122021_choreo1' '1p_Dirk_choreo1_04.12.2021'
 '1p_Earl_05122021_choreo1_flipped_lr' '1p_Eero_7_05122021_choreo1'
 '1p_Artur_05122021_chreo1' '1p_Stef_2_05122021_Choreo1'
 '1p_Eero_1_05122021_choreo1' '1p_Eero_6_05122021_choreo1'
 '1p_Heidi_07122021_choreo1_-turns' '1p_Raquelinha_05122021_choreo1']


In [225]:

# Normalize only X  clip by clip

df_perspective_n = df_perspective1[pose_x_list].copy()   
for clip_name in clip_names:
  df_perspective_new = pd.DataFrame(normalize(df_perspective_n.loc[df_perspective1['clip_name']==clip_name]))   # the Numpy Array is converted to a Pandas df. no column names.
  df_perspective_n.update(df_perspective_new)                 

#[df_perspective1["clip_name"==clip_name]]
df_perspective_n.columns = pose_x_list                    # adds columnnames
df_perspective_n # a numpy array

df_perspective1_norm = df_perspective1.copy()
df_perspective1_norm.update(df_perspective_n)






In [229]:
## Saving the results

data_output_csv = os.path.join(data_output_path, "Data_train_perspective1.csv")
df_perspective1_norm.to_csv(data_output_csv)



In [ ]:
df_perspective1.describe()

In [206]:
df_original.describe()

,Unnamed: 0,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,figure_id
count,35922.000000,35922.000000,35922.0,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000
mean,17960.500000,2176.194505,0.0,0.533765,0.450161,0.495789,0.502121,0.494658,0.495463,0.497842,0.562935,0.491151,0.534129,0.522524,0.508624,0.525366,0.599740,0.526724,0.494982,0.488097,0.387175,0.485306,0.390143,0.504955,0.411212,0.497696,0.532416,0.523706,0.396818,0.512566,0.395166,0.494458,0.545189,0.496793,0.473746,0.552732,0.439472,0.485432,0.502263,0.534385,0.495762,0.506684,0.517889,0.509481,0.514551,0.491802,0.571052,0.502020,0.538752,0.506921,0.537916,0.499661,0.553511,0.670343,0.832626,0.800229,0.727318,0.668552,0.805106,0.728831,0.656387,0.696712,0.673130,0.789598,0.764287,0.668011,0.787115,0.751815,0.467002,0.488698,0.581164,0.610187,0.571973,0.571232,0.638389,0.595156,0.578432,0.648212,238.371221
std,10369.932521,1269.999791,0.0,0.199172,0.238272,0.199652,0.195404,0.186996,0.185703,0.204392,0.169782,0.194664,0.173257,0.214594,0.193916,0.216464,0.165579,0.201547,0.166217,0.187437,0.173533,0.177140,0.174837,0.163568,0.173849,0.165555,0.199002,0.202022,0.179594,0.194800,0.182652,0.185529,0.204999,0.222392,0.272523,0.196695,0.252061,0.205238,0.200492,0.217555,0.214985,0.174887,0.219204,0.178263,0.222015,0.187868,0.206189,0.161883,0.208882,0.163497,0.211578,0.170535,0.203631,0.385498,0.092733,0.108774,0.267517,0.303770,0.105046,0.267377,0.305226,0.094135,0.111150,0.078152,0.136665,0.114205,0.083231,0.141426,0.435656,0.439764,0.395762,0.387755,0.261378,0.272148,0.202702,0.251557,0.254387,0.194085,134.187041
min,0.000000,37.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.268592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8980.250000,1155.250000,0.0,0.399662,0.258982,0.355473,0.357400,0.360215,0.359450,0.342777,0.441246,0.352072,0.427984,0.

In [207]:
df_perspective1_norm.describe()

,Unnamed: 0,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,figure_id
count,35922.000000,35922.000000,35922.0,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000,35922.000000
mean,17960.500000,2176.194505,0.0,0.493244,0.450161,0.482459,0.502121,0.481613,0.495463,0.482338,0.562935,0.490877,0.534129,0.491156,0.508624,0.481653,0.599740,0.471161,0.494982,0.475561,0.387175,0.478623,0.390143,0.481525,0.411212,0.483892,0.532416,0.471311,0.396818,0.478988,0.395166,0.479394,0.545189,0.495128,0.473746,0.486068,0.439472,0.495947,0.502263,0.480288,0.495762,0.484235,0.517889,0.480309,0.514551,0.479638,0.571052,0.479414,0.538752,0.482913,0.537916,0.482709,0.553511,0.670343,0.832626,0.800229,0.727318,0.668552,0.805106,0.728831,0.656387,0.696712,0.673130,0.789598,0.764287,0.668011,0.787115,0.751815,0.467002,0.488698,0.581164,0.610187,0.571973,0.571232,0.638389,0.595156,0.578432,0.648212,238.371221
std,10369.932521,1269.999791,0.0,0.175638,0.238272,0.171545,0.195404,0.172788,0.185703,0.165599,0.169782,0.168813,0.173257,0.172515,0.193916,0.169436,0.165579,0.168073,0.166217,0.167798,0.173533,0.167580,0.174837,0.168447,0.173849,0.167009,0.199002,0.168032,0.179594,0.172545,0.182652,0.174603,0.204999,0.175658,0.272523,0.173391,0.252061,0.174401,0.200492,0.170658,0.214985,0.174288,0.219204,0.176738,0.222015,0.173608,0.206189,0.164456,0.208882,0.164628,0.211578,0.167539,0.203631,0.385498,0.092733,0.108774,0.267517,0.303770,0.105046,0.267377,0.305226,0.094135,0.111150,0.078152,0.136665,0.114205,0.083231,0.141426,0.435656,0.439764,0.395762,0.387755,0.261378,0.272148,0.202702,0.251557,0.254387,0.194085,134.187041
min,0.000000,37.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.268592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8980.250000,1155.250000,0.0,0.378910,0.258982,0.376934,0.357400,0.374248,0.359450,0.375885,0.441246,0.383330,0.427984,0.